# Bônus

Análise de variáveis utilizando o modelo de regressão logística.

Nossa intenção, com essa análise, é identificar as variáveis que influenciam na saída, ou não, de um colaborador. Uma vez identificadas as variáveis relevantes, a empresa poderia traçar um plano de ação em cima delas, que visasse reter talentos e evitar um grande turnover.

### Carregar os pacotes necessários

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
# Carregando o banco de dados do Github

url = 'https://raw.github.com/anacarolpimenta/Desafio-Rocketmat/master/HR_Engagement_Sat_Sales_UpdatedV4.0.csv'
db2 = pd.read_csv(url, sep = ",", usecols = ['Role','Percent_Remote',
                'last_evaluation','number_project','time_spend_company',
                'salary','LinkedIn_Hits','promotion_last_5years',
                'Emp_Title','Emp_Role','Emp_Position','average_montly_hours',
                "left_Company"]) 

db2.columns = db2.columns.str.lower() # Mudando todas as letras para minusculas
for columns in db2.columns:
  print(columns)

role
percent_remote
last_evaluation
number_project
average_montly_hours
time_spend_company
left_company
promotion_last_5years
salary
linkedin_hits
emp_role
emp_position
emp_title


Para a Regressão Logística, iremos trabalhar com o banco de dados já pré-selecionado (durante o processo do modelo de redes neurais).



Antes de realizar a Regressão, irei, novamente, recodificar as variáveis. Para a regressão logística, trabalharemos com as seguintes variávels independentes e dependente:


**Variável Dependente**: 
> "left_company" (sair da empresa)
>> Transformarei a variável em categórica (0 e 1);


**Variáveis Independentes**:
*   "role" - Papel na Empresa (*categórica*)
*   "percent_remote" - Percentual do Trabalho feito de Home Office (*categórica*)
*   "last_evaluation" - Resultado na última avaliação (*contínua*)
*   "number_project" - Número de projetos (*discreta*)
*   "average_montly_hours" - Média de horas trabalhadas por mês (*discreta*)
*   "time_spend_company" - Anos na empresa (*discreta*)
*   "promotion_last_5years" - Se foi promovido nos últimos 5 anos (*categórica*)
*   "salary" - Salário (*categórica*)
*   "linkedin_hits" - Número de vezes que acessa o LinkedIn no computador da empresa (*contínua*)
*   "emp_role" - Como o trabalhador se identifica com a importância da sua função na empresa (*categórica/ordinal*)
*   "emp_position" - Como o trabalhador se identifica com a importância da sua posição na empresa (*categórica/ordinal*)
*   "emp_title" - Como o trabalhador se sente quanto a seu "título" na empresa (*categórica/ordinal*)





### 1) Tratar as variáveis que serão usadas:

Aqui irei tratar as variáveis, fazer as recodificações necessárias. Preparar os dados para a Regressão


#### Variável Dependente:

In [ ]:
## Variável Dependente:
db2["left_company_cat"] = db2["left_company"].astype("category")

#### Variáveis Independentes:

In [ ]:

## Variável Independente:
### "role":
sorted(db2["role"].unique().tolist()) # Verificando os valores únicos:

# Reordenar nossa variável "role" do menor nível para o maior:
niveis = {'Director': "5 - Director",
          'Level 1': "1 - Level 1",
          'Level 2-4':"2 - Level 2-4",
          'Manager': "3 - Manager",
          'Senior Director': "6 - Senior Director",
          'Senior Manager': "4 - Senior manager",
          'VP': "7 - VP"}
db2['new_role'] = db2['role'].map(niveis) # Recodificando a variável
sorted(db2["new_role"].unique()) # Conferindo se foi feita a mudança

db2["new_role"] = db2["new_role"].astype("category") # Transformando em variável categórica;


In [ ]:
### "salary"

sorted(db2["salary"].unique().tolist()) # Conferindo os valores únicos

# Reordenando os níveis do mais baixo para o mais alto
nivel_sal = {'high': "3 - Alto", 'low': "1 - Baixo", 'medium': "2 - Medio"}
db2["new_sal"] = db2["salary"].map(nivel_sal)
sorted(db2["new_sal"].unique().tolist()) # Conferindo as alterações

db2["new_sal"] = db2["new_sal"].astype("category") # Transformando em variável categórica;


In [ ]:
### "percent_remote"

sorted(db2["percent_remote"].unique().tolist()) # Conferindo os valores únicos

nivel_percent = {0.4: "1 - 40%", 0.5: "2 - 50%", 0.8: "3 - 80%", 1:"4 - 100%"} # Estabelecendo novos valores;
db2["new_percent_remote"] = db2["percent_remote"].map(nivel_percent) 
sorted(db2["new_percent_remote"].unique().tolist()) # Conferindo as alterações feitas;


db2["new_percent_remote"] = db2["new_percent_remote"].astype("category") # Transformando em categórica

In [ ]:
### "promotion_last_5years"

print(sorted(db2["promotion_last_5years"].unique().tolist()))

print(pd.crosstab(index = db2["promotion_last_5years"], columns = "Contagem"))
### A partir da tabela, interpretarei o 0 como sendo "não recebeu" e o 1 como sendo "recebeu"
#### Aqui quero identificar o impacto de NÃO ter recebido promoção

# Criando uma função para possibilitar recodificar a variável:
def prom(x): 
  if x == 1:
    return 0
  elif x == 0:
    return 1

# Aplicando a função:
db2["prom_last_5y"] = db2["promotion_last_5years"].apply(prom)

print("_" * 30) # Linha para separar os "prints" anteriores
print(pd.crosstab(index = db2["prom_last_5y"], columns = "Contagem")) # Tabela para conferir;

db2["prom_last_5y"] = db2["prom_last_5y"].astype("category") # Transformando em categórica

[0, 1]
col_0                  Contagem
promotion_last_5years          
0                         14680
1                           319
______________________________
col_0         Contagem
prom_last_5y          
0                  319
1                14680


### 2) Criando o Modelo de Regressão Logística

In [ ]:
# Criando o "modelo", que receberá a formula, as variáveis e depois iremos roda-lo;
modelo = smf.glm(formula = 'left_company_cat ~ new_role + new_percent_remote + last_evaluation + number_project + average_montly_hours + time_spend_company + prom_last_5y + new_sal + linkedin_hits + emp_role + emp_position + emp_title',
                 data = db2, family = sm.families.Binomial()).fit()

Com o modelo criado e treinado, agora iremos ver um "summary" para verificar quais variáveis se mostraram estatisticamente significantes para prever a saída do funcionário da empresa;

In [ ]:
print(modelo.summary())

                               Generalized Linear Model Regression Results                                
Dep. Variable:     ['left_company_cat[0]', 'left_company_cat[1]']   No. Observations:                14999
Model:                                                        GLM   Df Residuals:                    14978
Model Family:                                            Binomial   Df Model:                           20
Link Function:                                              logit   Scale:                          1.0000
Method:                                                      IRLS   Log-Likelihood:                -4834.8
Date:                                            Mon, 10 Aug 2020   Deviance:                       9669.6
Time:                                                    00:11:02   Pearson chi2:                 1.83e+04
No. Iterations:                                                21                                         
Covariance Type:                     

Aqui, se analisarmos a coluna "P>|z|" podemos ver quais variáveis foram estatisticamente significativas, com um p-valor abaixo de 0.05, adotando 95% de confiança. Abaixo trago as variáveis que ***NÃO*** foram significativas:



*   "new_role" - Variável "role": Cargo na empresa
*   "last_evaluation": Ultim avaliação;
*   "promotion_last_5years": Foi promovido nos ultimos 5 anos;
*   "linkedin_hits": Quantas vezes acessa o linkedin no computador da empresa



In [ ]:
# Aqui irei trazer a exponencial dos valores (para podermos ter a razão de chance), subtraida de 1 e multiplicada por 100:
# Assim, temos a porcentagem dos valores
(np.exp(modelo.params[1:]) - 1) * 100

new_role[T.2 - Level 2-4]            -2.723162
new_role[T.3 - Manager]               6.154275
new_role[T.4 - Senior manager]        6.994010
new_role[T.5 - Director]             -8.166493
new_role[T.6 - Senior Director]      -5.849770
new_role[T.7 - VP]                  -42.630885
new_percent_remote[T.2 - 50%]      1759.280727
new_percent_remote[T.3 - 80%]       -98.070485
new_percent_remote[T.4 - 100%]     -100.000000
prom_last_5y[T.1]                   -14.303122
new_sal[T.2 - Medio]                 36.737781
new_sal[T.3 - Alto]                 408.877527
last_evaluation                      91.431806
number_project                       72.846917
average_montly_hours                  0.256814
time_spend_company                  -12.678104
linkedin_hits                         0.005441
emp_role                            349.874724
emp_position                        304.229006
emp_title                           297.976820
dtype: float64

Com as informações acima, poderiamos supor algumas coisas. Abaixo trago alguns exemplos que poderiam ser analisados para traçar um plano de ação. Levando em consideração que as informações do Banco são incompletas, faltam descrições e, em muitos casos, não condizem com a realidade (como mostrado na seção de Análise):


1.  **O aumento da porcentagem de trabalho feito remotamente diminui a chance de sair da empresa**: Isso poderia estar relacionado ao fato de a pessoa gastar menos tempo no trânsito, ter mais flexibilidade para a resolução de problemas pessoais e mais comodidade. Ao evitar perder tempo no trânsito, é possível otimizar seu tempo e aumentar sua produtividade.

2.  **O aumento no número de projetos no qual as pessoas trabalham, influenciam em sua saída**: Isso pode ser justificado por uma possível sobrecarga.

3.  **O sentimento em relação a importância do seu papel, trabalho e título na empresa, podem influênciar na decisão de sair**:
> As variáveis *emp_role\position\title* parecem estar distribuídas de acordo com uma escala Likert de 5 pontos. No entanto, a falta de descrição sobre sua construção, aliado ao resultado da regressão, me fizeram interpreta-la da seguinte maneira:
1 - Sentimentos mais Positivos; 5 - Sentimentos mais Negativos
